In [1]:
import numpy as np

In [2]:
print("="*60)
print("1. ПОДГОТОВКА РАЗЛИЧНЫХ ВХОДНЫХ ДАННЫХ")
print("="*60)

def create_diagonally_dominant_matrix(n):
    A = np.random.rand(n, n) * 2 - 1
    for i in range(n):
        row_sum = np.sum(np.abs(A[i])) - np.abs(A[i, i])
        A[i, i] = row_sum + np.random.uniform(1, 3)
    return A

def create_weakly_convergent_matrix(n):
    A = np.eye(n) * 2
    for i in range(n):
        for j in range(n):
            if i != j:
                A[i, j] = np.random.uniform(0.01, 0.05)
    return A

def prepare_system(A, b):
    n = len(b)
    B = np.zeros((n, n))
    c = np.zeros(n)
    for i in range(n):
        if abs(A[i, i]) < 1e-12:
            raise ValueError(f"Нулевой диагональный элемент a_{i+1}{i+1}")
        for j in range(n):
            if i == j:
                B[i, j] = 0.0
            else:
                B[i, j] = -A[i, j] / A[i, i]
        c[i] = b[i] / A[i, i]
    return B, c

np.random.seed(42)
n = 5
print("\nа) Система 1: Быстрая сходимость (диагональное преобладание)")
A1 = np.array([
    [10.0, -1.0, 2.0, 0.0, 1.0],
    [-1.0, 11.0, -1.0, 3.0, 2.0],
    [2.0, -1.0, 10.0, -1.0, 3.0],
    [0.0, 3.0, -1.0, 8.0, 1.0],
    [1.0, 2.0, 3.0, 1.0, 15.0]
])
b1 = np.array([6.0, 25.0, -11.0, 15.0, 22.0])
B1, c1 = prepare_system(A1, b1)
print(f"Спектральный радиус ρ(B) = {np.max(np.abs(np.linalg.eigvals(B1))):.4f}")
print("\nб) Система 2: Медленная сходимость")
A2 = create_weakly_convergent_matrix(n)
b2 = np.random.randn(n)
B2, c2 = prepare_system(A2, b2)
print(f"Спектральный радиус ρ(B) = {np.max(np.abs(np.linalg.eigvals(B2))):.4f}")
print("\nв) Система 3: Случайная матрица")
A3 = create_diagonally_dominant_matrix(n)
b3 = np.random.randn(n)
B3, c3 = prepare_system(A3, b3)
rho3 = np.max(np.abs(np.linalg.eigvals(B3)))
print(f"Спектральный радиус ρ(B) = {rho3:.4f}")

1. ПОДГОТОВКА РАЗЛИЧНЫХ ВХОДНЫХ ДАННЫХ

а) Система 1: Быстрая сходимость (диагональное преобладание)
Спектральный радиус ρ(B) = 0.4269

б) Система 2: Медленная сходимость
Спектральный радиус ρ(B) = 0.0566

в) Система 3: Случайная матрица
Спектральный радиус ρ(B) = 0.2708


In [3]:
print("\n" + "="*60)
print("2. ВЫБОР НАЧАЛЬНЫХ ПРИБЛИЖЕНИЙ")
print("="*60)
initial_approximations = {
    "Нулевой вектор": np.zeros(n),
    "Вектор из единиц": np.ones(n),
    "Правая часть": b1.copy(),
    "Случайный вектор": np.random.randn(n)
}
print("\nНачальные приближения:")
for name, vector in initial_approximations.items():
    print(f"{name}: {np.round(vector, 3)}")


2. ВЫБОР НАЧАЛЬНЫХ ПРИБЛИЖЕНИЙ

Начальные приближения:
Нулевой вектор: [0. 0. 0. 0. 0.]
Вектор из единиц: [1. 1. 1. 1. 1.]
Правая часть: [  6.  25. -11.  15.  22.]
Случайный вектор: [-0.461  1.057  0.344 -1.763  0.324]


In [4]:
print("\n" + "="*60)
print("3. РЕАЛИЗАЦИЯ МЕТОДА ПРОСТОЙ ИТЕРАЦИИ")
print("="*60)

def simple_iteration(B, c, x0, eps=1e-6, max_iter=1000, criterion="difference", B_norm=None):
    if B_norm is None:
        B_norm = np.linalg.norm(B, ord=np.inf)  # используем бесконечную норму
    x = x0.copy()
    history = [x.copy()]
    errors = []
    print(f"\nНачало итераций с критерием: {criterion}")
    print(f"Начальное приближение: {np.round(x0, 4)}")
    print(f"Норма матрицы B: {B_norm:.4f}")
    print(f"Спектральный радиус ρ(B): {np.max(np.abs(np.linalg.eigvals(B))):.4f}")
    for k in range(max_iter):
        x_new = B @ x + c
        history.append(x_new.copy())
        diff = np.linalg.norm(x_new - x, ord=np.inf)
        if criterion == "aposteriori" and B_norm < 1:
            error_est = (B_norm / (1 - B_norm)) * diff
        elif criterion == "residual":
            residual = np.linalg.norm((np.eye(len(B)) - B) @ x_new - c, ord=np.inf)
            error_est = residual
        else:
            error_est = diff
        errors.append(error_est)
        if k < 5 or (k+1) % 10 == 0:
            print(f"Итерация {k+1:3d}: x = {np.round(x_new, 6)}, оценка ошибки = {error_est:.2e}")
        if error_est < eps:
            print(f"\nДостигнута точность {eps} за {k+1} итераций")
            print(f"Финальное приближение: {np.round(x_new, 6)}")
            return x_new, k+1, history, errors
        x = x_new
    print(f"\nДостигнут предел итераций ({max_iter}) без достижения точности")
    print(f"Текущая оценка ошибки: {errors[-1]:.2e}")
    return x, max_iter, history, errors

print("\nТестирование на Системе 1 (быстрая сходимость):")
A = A1
b = b1
B = B1
c = c1
B_norm = np.linalg.norm(B, ord=np.inf)
x_exact = np.linalg.solve(A, b)
print(f"\nТочное решение: {np.round(x_exact, 6)}")
results = {}
for name, x0 in initial_approximations.items():
    print(f"\n{'='*40}")
    print(f"Начальное приближение: {name}")
    x_approx, iterations, history, errors = simple_iteration(
        B, c, x0, eps=1e-6, max_iter=100,
        criterion="difference", B_norm=B_norm
    )
    real_error = np.linalg.norm(x_approx - x_exact, ord=np.inf)
    print(f"Реальная ошибка: {real_error:.2e}")
    results[name] = {
        'iterations': iterations,
        'final_error': real_error,
        'history': history,
        'errors': errors
    }


3. РЕАЛИЗАЦИЯ МЕТОДА ПРОСТОЙ ИТЕРАЦИИ

Тестирование на Системе 1 (быстрая сходимость):

Точное решение: [ 0.921888  1.732595 -1.445865  0.868863  1.405444]

Начальное приближение: Нулевой вектор

Начало итераций с критерием: difference
Начальное приближение: [0. 0. 0. 0. 0.]
Норма матрицы B: 0.7000
Спектральный радиус ρ(B): 0.4269
Итерация   1: x = [ 0.6       2.272727 -1.1       1.875     1.466667], оценка ошибки = 2.27e+00
Итерация   2: x = [ 0.900606  1.449242 -1.245227  0.701894  1.218636], оценка ошибки = 1.17e+00
Итерация   3: x = [ 0.872106  1.828402 -1.430598  1.023551  1.415646], оценка ошибки = 3.79e-01
Итерация   4: x = [ 0.927395  1.685415 -1.41392   0.833569  1.382622], оценка ошибки = 1.90e-01
Итерация   5: x = [ 0.913063  1.749775 -1.448367  0.893402  1.407331], оценка ошибки = 6.44e-02
Итерация  10: x = [ 0.921991  1.732346 -1.445699  0.868613  1.405395], оценка ошибки = 9.40e-04

Достигнута точность 1e-06 за 18 итераций
Финальное приближение: [ 0.921888  1.732595 -1.4

In [5]:
print("\n" + "="*60)
print("4. ТРЕБУЕМАЯ ТОЧНОСТЬ: 10^-3 И 10^-6")
print("="*60)

epsilons = [1e-3, 1e-6]

for eps in epsilons:
    print(f"\nТочность ε = {eps}")
    print("-" * 40)
    x0 = np.zeros(n)
    for criterion in ["difference", "aposteriori"]:
        print(f"\nКритерий остановки: {criterion}")
        x_approx, iterations, _, _ = simple_iteration(
            B, c, x0, eps=eps, max_iter=1000,
            criterion=criterion, B_norm=B_norm
        )
        real_error = np.linalg.norm(x_approx - x_exact, ord=np.inf)
        print(f"Итераций: {iterations}, реальная ошибка: {real_error:.2e}")


4. ТРЕБУЕМАЯ ТОЧНОСТЬ: 10^-3 И 10^-6

Точность ε = 0.001
----------------------------------------

Критерий остановки: difference

Начало итераций с критерием: difference
Начальное приближение: [0. 0. 0. 0. 0.]
Норма матрицы B: 0.7000
Спектральный радиус ρ(B): 0.4269
Итерация   1: x = [ 0.6       2.272727 -1.1       1.875     1.466667], оценка ошибки = 2.27e+00
Итерация   2: x = [ 0.900606  1.449242 -1.245227  0.701894  1.218636], оценка ошибки = 1.17e+00
Итерация   3: x = [ 0.872106  1.828402 -1.430598  1.023551  1.415646], оценка ошибки = 3.79e-01
Итерация   4: x = [ 0.927395  1.685415 -1.41392   0.833569  1.382622], оценка ошибки = 1.90e-01
Итерация   5: x = [ 0.913063  1.749775 -1.448367  0.893402  1.407331], оценка ошибки = 6.44e-02
Итерация  10: x = [ 0.921991  1.732346 -1.445699  0.868613  1.405395], оценка ошибки = 9.40e-04

Достигнута точность 0.001 за 10 итераций
Финальное приближение: [ 0.921991  1.732346 -1.445699  0.868613  1.405395]
Итераций: 10, реальная ошибка: 2.50e-0

In [6]:
print("\n" + "="*60)
print("5. СРАВНЕНИЕ РАЗЛИЧНЫХ КРИТЕРИЕВ ОКОНЧАНИЯ")
print("="*60)

def test_criteria(B, c, x0, eps=1e-6, B_norm=None):
    if B_norm is None:
        B_norm = np.linalg.norm(B, ord=np.inf)
    criteria_results = {}
    for criterion_name in ["difference", "aposteriori", "residual"]:
        print(f"\nКритерий: {criterion_name}")
        if criterion_name == "aposteriori" and B_norm >= 1:
            print("  Неприменим: ||B|| >= 1")
            continue
        x_approx, iterations, _, errors = simple_iteration(
            B, c, x0, eps=eps, max_iter=1000,
            criterion=criterion_name, B_norm=B_norm
        )
        real_error = np.linalg.norm(x_approx - x_exact, ord=np.inf)
        criteria_results[criterion_name] = {
            'iterations': iterations,
            'real_error': real_error,
            'last_estimated_error': errors[-1] if errors else None
        }
    return criteria_results
print("\nСравнение критериев для Системы 1 (||B|| = {:.4f}):".format(B_norm))
criteria_results1 = test_criteria(B1, c1, np.zeros(n), eps=1e-6, B_norm=B_norm)
print("\n\nСравнение критериев для Системы 2:")
B2_norm = np.linalg.norm(B2, ord=np.inf)
print(f"||B|| = {B2_norm:.4f}")
criteria_results2 = test_criteria(B2, c2, np.zeros(n), eps=1e-3, B_norm=B2_norm)
print("\nОбоснование выбора критериев:")
print("1. Критерий разности (|x^(m) - x^(m-1)| < ε):")
print("   - Простейший в реализации")
print("   - Обоснован при ||B|| ≤ 1/2")
print("   - Может давать ложную сходимость при медленной сходимости")

print("\n2. Апостериорный критерий (оценка (16)):")
print("   - Более надежный")
print("   - Требует знания ||B||")
print("   - Неприменим при ||B|| ≥ 1")

print("\n3. Критерий невязки:")
print("   - Основан на невязке ||x - Bx - c||")
print("   - Работает всегда, но может быть менее точным")


5. СРАВНЕНИЕ РАЗЛИЧНЫХ КРИТЕРИЕВ ОКОНЧАНИЯ

Сравнение критериев для Системы 1 (||B|| = 0.7000):

Критерий: difference

Начало итераций с критерием: difference
Начальное приближение: [0. 0. 0. 0. 0.]
Норма матрицы B: 0.7000
Спектральный радиус ρ(B): 0.4269
Итерация   1: x = [ 0.6       2.272727 -1.1       1.875     1.466667], оценка ошибки = 2.27e+00
Итерация   2: x = [ 0.900606  1.449242 -1.245227  0.701894  1.218636], оценка ошибки = 1.17e+00
Итерация   3: x = [ 0.872106  1.828402 -1.430598  1.023551  1.415646], оценка ошибки = 3.79e-01
Итерация   4: x = [ 0.927395  1.685415 -1.41392   0.833569  1.382622], оценка ошибки = 1.90e-01
Итерация   5: x = [ 0.913063  1.749775 -1.448367  0.893402  1.407331], оценка ошибки = 6.44e-02
Итерация  10: x = [ 0.921991  1.732346 -1.445699  0.868613  1.405395], оценка ошибки = 9.40e-04

Достигнута точность 1e-06 за 18 итераций
Финальное приближение: [ 0.921888  1.732595 -1.445865  0.868863  1.405443]

Критерий: aposteriori

Начало итераций с критерие

In [7]:
print("\n" + "="*60)
print("6. ОЦЕНКА ЧИСЛА АРИФМЕТИЧЕСКИХ ОПЕРАЦИЙ")
print("="*60)

def estimate_operations(n, iterations, method="simple_iteration"):
    if method == "simple_iteration":
        multiplications_per_iter = n * n  # B * x
        additions_per_iter = n * (n - 1) + n
        total_per_iter = multiplications_per_iter + additions_per_iter
        total_ops = iterations * total_per_iter
        print(f"\nОценка для системы {n}×{n}, {iterations} итераций:")
        print(f"  Умножений на итерацию: {multiplications_per_iter}")
        print(f"  Сложений на итерацию: {additions_per_iter}")
        print(f"  Всего операций на итерацию: {total_per_iter}")
        print(f"  Общее число операций: {total_ops:.0f}")
        return total_ops


print("Пример оценки операций:")
estimate_operations(5, 10)
print("\nСравнение с методом Гаусса:")
n = 5
iterations_needed = 15
simple_iter_ops = estimate_operations(n, iterations_needed)
gauss_ops = (2/3) * n**3 + 2 * n**2
print(f"\nМетод Гаусса (LU-разложение + решение):")
print(f"  LU-разложение: ~{(2/3)*n**3:.0f} операций")
print(f"  Решение треугольных систем: ~{2*n**2:.0f} операций")
print(f"  Всего: ~{gauss_ops:.0f} операций")
if 'simple_iter_ops' in locals():
    ratio = gauss_ops / simple_iter_ops
    print(f"\nОтношение операций (Гаусс / Простая итерация): {ratio:.2f}")
    if ratio > 1:
        print(f"Метод Гаусса требует в {ratio:.1f} раз больше операций")
    else:
        print(f"Метод простой итерации требует в {1/ratio:.1f} раз больше операций")


6. ОЦЕНКА ЧИСЛА АРИФМЕТИЧЕСКИХ ОПЕРАЦИЙ
Пример оценки операций:

Оценка для системы 5×5, 10 итераций:
  Умножений на итерацию: 25
  Сложений на итерацию: 25
  Всего операций на итерацию: 50
  Общее число операций: 500

Сравнение с методом Гаусса:

Оценка для системы 5×5, 15 итераций:
  Умножений на итерацию: 25
  Сложений на итерацию: 25
  Всего операций на итерацию: 50
  Общее число операций: 750

Метод Гаусса (LU-разложение + решение):
  LU-разложение: ~83 операций
  Решение треугольных систем: ~50 операций
  Всего: ~133 операций

Отношение операций (Гаусс / Простая итерация): 0.18
Метод простой итерации требует в 5.6 раз больше операций


In [8]:
print("\n" + "="*60)
print("ДОПОЛНИТЕЛЬНЫЙ АНАЛИЗ СХОДИМОСТИ")
print("="*60)

def analyze_convergence(B, c, x0, x_exact, max_iter=50):
    x = x0.copy()
    errors = []
    print("\nИтерация | Ошибка ||x - x*||∞ | Отношение ошибок")
    print("-" * 50)
    prev_error = None
    for k in range(max_iter):
        x = B @ x + c
        error = np.linalg.norm(x - x_exact, ord=np.inf)
        errors.append(error)
        if prev_error is not None and prev_error > 0:
            ratio = error / prev_error
            print(f"{k+1:8d} | {error:15.2e} | {ratio:15.4f}")
        else:
            print(f"{k+1:8d} | {error:15.2e} | {'-':>15}")
        prev_error = error
    if len(errors) > 5:
        last_errors = errors[-5:]
        ratios = [last_errors[i+1]/last_errors[i] for i in range(len(last_errors)-1)]
        avg_ratio = np.mean(ratios)
        print(f"\nСреднее отношение ошибок (последние 5 итераций): {avg_ratio:.4f}")
        print(f"Спектральный радиус ρ(B): {np.max(np.abs(np.linalg.eigvals(B))):.4f}")
print("\nАнализ сходимости для Системы 1:")
analyze_convergence(B1, c1, np.zeros(n), x_exact, max_iter=20)
print("\n" + "="*60)
print("ВЫВОДЫ:")
print("="*60)
print("1. Метод простой итерации сходится при ρ(B) < 1")
print("2. Скорость сходимости определяется спектральным радиусом ρ(B)")
print("3. Начальное приближение влияет на число итераций, но не на сходимость")
print("4. Критерий остановки должен выбираться с учетом ||B||:")
print("   - При ||B|| < 1/2 можно использовать простой критерий разности")
print("   - При ||B|| близком к 1 нужен апостериорный критерий")
print("5. Для плотных матриц метод простой итерации требует O(n²) операций на итерацию")
print("6. По сравнению с прямыми методами (Гаусс) итерационные методы")
print("   эффективны для больших разреженных систем")


ДОПОЛНИТЕЛЬНЫЙ АНАЛИЗ СХОДИМОСТИ

Анализ сходимости для Системы 1:

Итерация | Ошибка ||x - x*||∞ | Отношение ошибок
--------------------------------------------------
       1 |        1.01e+00 |               -
       2 |        2.83e-01 |          0.2816
       3 |        1.55e-01 |          0.5459
       4 |        4.72e-02 |          0.3050
       5 |        2.45e-02 |          0.5201
       6 |        8.07e-03 |          0.3287
       7 |        4.05e-03 |          0.5022
       8 |        1.41e-03 |          0.3473
       9 |        6.90e-04 |          0.4904
      10 |        2.50e-04 |          0.3621
      11 |        1.20e-04 |          0.4810
      12 |        4.62e-05 |          0.3847
      13 |        2.13e-05 |          0.4598
      14 |        8.50e-06 |          0.3999
      15 |        3.80e-06 |          0.4471
      16 |        1.56e-06 |          0.4097
      17 |        6.84e-07 |          0.4394
      18 |        2.85e-07 |          0.4160
      19 |        1.2